In [24]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [25]:
#Creating a base map
m = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
#m

In [26]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [27]:
covid1= json.loads(data)

In [28]:
pd.json_normalize(covid1['Countries'],sep=",");

In [29]:
df = pd.DataFrame(covid1['Countries'])
#df

In [30]:
covid2 = df.drop(columns =['CountryCode','Slug','Date','Premium'],axis=1) 

In [31]:
# World countries geodata
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [32]:
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    min_zoom=2,
    name='COVID-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='grey',
    legend_name='Total Confirmed Covid Cases',
).add_to(m);
#m

In [33]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [34]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
#coordinates

In [35]:
covid_final= pd.merge(covid2,coordinates,on='Country')

In [36]:
def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)


In [37]:
#use df.apply(,axis=1) to iterate through every row in your dataframe
covid_final.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

#m


In [38]:
m1 = folium.Map(tiles='StamenToner', min_zoom=1.5)

#m1

In [39]:
deaths=covid_final['TotalDeaths'].astype(float)

In [40]:
lat=covid_final['latitude'].astype(float)

In [41]:
lon=covid_final['longitude'].astype(float)

In [42]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0));

In [43]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)

#use df.apply(,axis=1) to iterate through every row in your dataframe
covid_final.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
m1.fit_bounds(m1.get_bounds())

#m1